In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import os,sys,random,time
import argparse
import numpy as np

from meld_graph.training import calculate_loss

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


In [2]:
# # create dummy estimates
# x = np.array([[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1], [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]]).T
# x= torch.from_numpy(x)
# print(x.shape)
# print(x)

In [3]:
# create dummy estimates
x = np.array([[0.1, 0.9, 0.1, 0.1, 0.9, 0.1, 0.9, 0.1, 0.9, 0.1, 0.9, 0.9, 0.9], 
              [0.9, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.9, 0.1, 0.9, 0.1, 0.1, 0.1]]).T
x= torch.from_numpy(x)
print(x.shape)
print(x)

torch.Size([13, 2])
tensor([[0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.1000, 0.9000],
        [0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.1000, 0.9000],
        [0.9000, 0.1000],
        [0.9000, 0.1000],
        [0.9000, 0.1000]], dtype=torch.float64)


In [4]:
# create dummy ground truth class
l_perfect = np.array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1])
l_perfect = torch.from_numpy(l_perfect)
print(l_perfect.shape)
print(l_perfect)

torch.Size([13])
tensor([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1])


In [5]:
# create dummy ground truth class
l_reverse = np.array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0])
l_reverse = torch.from_numpy(l_reverse)
print(l_reverse.shape)
print(l_reverse)

torch.Size([13])
tensor([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0])


In [6]:
params={'loss_dictionary': { 
                            'focal_loss':{'weight':1, 'gamma':0, 'alpha': 0.5},
              },
       }

In [7]:
params['loss_dictionary'].keys()

dict_keys(['focal_loss'])

In [8]:
loss_weight_dictionary = params['loss_dictionary']
estimates=x
labels = l_reverse

output0 = calculate_loss(loss_weight_dictionary,estimates,labels, device=None)
output1 = nn.CrossEntropyLoss()(estimates,labels)

/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/training.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = torch.nn.functional.log_softmax(inputs)


In [25]:
# x = Variable(x.cuda())
# print(x.shape)
# print(x)

In [9]:
output0

tensor(0.1998, dtype=torch.float64)

In [10]:
output1

tensor(0.3711, dtype=torch.float64)

### find proportion lesion/non-lesional vertices

In [1]:
from meld_classifier.paths import BASE_PATH, EXPERIMENT_PATH
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
import os
import numpy as np

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1
Setting FS_SUBJECTS_PATH to 


In [2]:
#load cohort
site_codes=['H2', 'H3','H4','H5','H6','H7','H9','H10','H11','H12','H14','H15','H16','H17','H18','H19',
                  'H21','H23','H24','H26']
c= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5', dataset=None)
listids = c.get_subject_ids(site_codes=site_codes, group='patient', lesional_only=True)

In [3]:
lesions=[]
for k,subject in enumerate(listids):
    subj = MeldSubject(subject, cohort=c)
    hemi = subj.get_lesion_hemisphere()
    if hemi:
        lesion = np.round(subj.load_feature_values(hemi=hemi, feature=".on_lh.lesion.mgh")[:])
        lesions.append(lesion)
lesions=np.array(lesions)

In [11]:
lesions= lesions.flatten()
lesional_v =lesions.sum()
print(lesional_v)
non_lesional_v = len(lesions)-lesional_v
print(non_lesional_v)
ratio = lesional_v/non_lesional_v
print(ratio)

1876315.0
87417575.0
0.02146381891742021


(163842,)